* 方針
    * なるべく


## preprocessing

In [1]:
import pandas as pd
import numpy as np

# csvファイルの読み込み
df = pd.read_csv('./train.csv', header=0) # 0行目がヘッダー(defaultの動作、指定しない場合はNoneを指定)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
#
#df["Age"].dropna().hist(bins=16, range=(0, 80), alpha=0.5)

In [3]:
# データ前処理
df.Embarked = df.Embarked.replace(['C', 'S', 'Q'], [0, 1, 2])
#df.Cabin = df.Cabin.replace('NaN', 0)
df.Sex = df.Sex.replace(['male', 'female'], [0, 1])
df.Age = df.Age.fillna(df.Age.median()) # とりあえず中央値で埋める df[df['Age'].isnull()] NaNを名前の敬称を元に割り当てた方がよさそう
df = df.dropna(subset=['Embarked']) # 乗船港が不明なのはどうしようもないので削る
#df

In [4]:
from sklearn import svm, datasets, cross_validation, preprocessing, grid_search
train_features = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked' ]].dropna().as_matrix()
train_labels = df['Survived'].as_matrix()

D:\Anaconda3\envs\py27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Anaconda3\envs\py27\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## fit

In [5]:
# http://aidiary.hatenablog.com/entry/20100503/1272889097
# http://tullo.ch/articles/svm-py/
# https://pythonprogramming.net/svm-in-python-machine-learning-tutorial/
# soft margin

In [6]:
# http://aidiary.hatenablog.com/entry/20100503/1272889097
import numpy as np
from scipy.linalg import norm
import cvxopt
import cvxopt.solvers
from pylab import *

C = 0.5         # スラック変数を用いて表されるペナルティとマージンのトレードオフパラメータ
SIGMA = 0.3     # ガウスカーネルのパラメータ

# 多項式カーネル
def polynomial_kernel(x, y):
    return (1 + np.dot(x, y)) ** P

# ガウスカーネル
def gaussian_kernel(x, y):
    return np.exp(-norm(x-y)**2 / (2 * (SIGMA ** 2)))

# どちらのカーネル関数を使うかここで指定
kernel = gaussian_kernel

# fit
X = train_features
t = train_labels * 2 - 1.0  # 教師信号を-1 or 1に変換
N = len(t)

# ラグランジュ乗数を二次計画法（Quadratic Programming）で求める
K = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        K[i, j] = t[i] * t[j] * kernel(X[i], X[j])

Q = cvxopt.matrix(K)
p = cvxopt.matrix(-np.ones(N))
temp1 = np.diag([-1.0]*N)
temp2 = np.identity(N)
G = cvxopt.matrix(np.vstack((temp1, temp2)))
temp1 = np.zeros(N)
temp2 = np.ones(N) * C
h = cvxopt.matrix(np.hstack((temp1, temp2)))
A = cvxopt.matrix(t, (1,N))
b = cvxopt.matrix(0.0)
sol = cvxopt.solvers.qp(Q, p, G, h, A, b)
a = array(sol['x']).reshape(N)
print a

     pcost       dcost       gap    pres   dres
 0: -3.0309e+02 -1.1202e+03  8e+03  5e+00  4e-16
 1: -2.1885e+02 -9.0716e+02  9e+02  2e-01  5e-16
 2: -2.3356e+02 -3.2110e+02  9e+01  1e-02  4e-16
 3: -2.5595e+02 -2.6873e+02  1e+01  2e-03  2e-16
 4: -2.5790e+02 -2.6112e+02  3e+00  4e-04  2e-16
 5: -2.5827e+02 -2.5956e+02  1e+00  1e-04  2e-16
 6: -2.5847e+02 -2.5872e+02  3e-01  2e-05  2e-16
 7: -2.5849e+02 -2.5864e+02  1e-01  5e-06  2e-16
 8: -2.5852e+02 -2.5855e+02  3e-02  9e-07  2e-16
 9: -2.5852e+02 -2.5853e+02  3e-03  8e-16  2e-16
10: -2.5852e+02 -2.5852e+02  4e-05  1e-14  2e-16
Optimal solution found.
[  3.95154634e-01   4.99999979e-01   4.99999962e-01   4.99999979e-01
   1.95218489e-01   3.71527945e-01   3.96791464e-01   3.96785534e-01
   4.99999979e-01   4.99999979e-01   4.99999979e-01   4.99999979e-01
   1.54388316e-01   3.95263410e-01   3.96799449e-01   4.99999979e-01
   3.96791463e-01   4.99999986e-01   3.96791464e-01   4.99999962e-01
   3.98017530e-01   4.99999988e-01   4.99999

In [7]:
# https://pythonprogramming.net/soft-margin-kernel-cvxopt-svm-machine-learning-tutorial/

## predict

In [8]:
# csvファイルの読み込み
df_test = pd.read_csv('./test.csv', header=0)

In [9]:
# データ前処理
df_test.Embarked = df_test.Embarked.replace(['C', 'S', 'Q'], [0, 1, 2])
#df.Cabin = df.Cabin.replace('NaN', 0)
df_test.Sex = df_test.Sex.replace(['male', 'female'], [0, 1])
df_test.Age = df_test.Age.fillna(df_test.Age.median())

In [10]:
df_test[df_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked' ]].isnull().any(1) == True]
df_test.Fare =df_test.Fare.fillna(df_test[df_test.Pclass == 3].Fare.median())# fareが不明なのが混ざっているので、とりあえずpcclass=3の中央値にしておく

In [11]:
# predict
test_features = df_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked' ]].as_matrix()
test = test_features

# サポートベクトルのインデックスを抽出
S = []
for n in range(len(a)):
    if a[n] < 1e-5: continue
    S.append(n)

def f(x, a, t, X, b):
    sum = 0.0
    for n in S:
        sum += a[n] * t[n] * kernel(x, X[n])
    return sum + b
    print sum + b
    
v = np.zeros(len(test))
for i in range(len(test)):
    v[i] = f(test[i], a, t, X, b)

test_pred = ((np.sign(v) + 1)/2).astype(np.int32)

In [12]:
# make submission data
df_csv = pd.concat([df_test.PassengerId, pd.DataFrame({'Survived':test_pred})], axis=1)
df_csv

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0
5,897,0
6,898,0
7,899,1
8,900,1
9,901,0


In [13]:
df_csv.to_csv('submission_scratch.csv', index=False)
# score=0.55981